In [3]:

import os
import sys
import random

import numpy as np
import pandas as pd
#import tensorflow as tf


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import cv2

#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from keras import layers
#from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from sklearn.model_selection import train_test_split
#from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.optimizers import SGD, schedules, Adam
#from keras.layers.convolutional import Conv2D, Conv2DTranspose
#from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
#from keras.layers.merge import concatenate, add
#from keras.models import Model, load_model
#from tensorflow.python.keras import losses

# Step 1: Load Dataset


In [4]:
image_height = 256
image_width = 256
dpath = "E:/Faculty of Computer and artifical intelligence/First semister 4/Image Proccesing/DataSet/archive/Dataset_BUSI_with_GT/"
classes = ['benign', 'malignant', 'normal']

In [5]:
data = {'image' : [],
        'mask' : []}


In [6]:
def load_data(path, data_obj, class_name):
    img_names_list = os.listdir(path+class_name)
    image_names = []
    mask_names = []
    names_truncated = []

    for i in range(len(img_names_list)):
        names_truncated.append(img_names_list[i].split(')')[0])

    names_truncated = list(set(names_truncated))

    for i in range(len(names_truncated)):
        image_names.append(names_truncated[i]+').png')
        mask_names.append(names_truncated[i]+')_mask.png')

    data_obj = preprocess_data(image_names, mask_names, image_width, image_height, path, class_name, data_obj)


    return data_obj


# Step 2: Data Preprocessing

In [7]:

def preprocess_data(image_names, mask_names, img_width, img_height, dpath, dclass, data_obj):
    for index in range (len(image_names)):

        image_path = dpath+'/'+dclass+'/'+ image_names[index]
        mask_path = dpath+'/'+dclass+'/'+ mask_names[index]

        x = cv2.imread(image_path, cv2.IMREAD_COLOR)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        x = np.round(cv2.resize(x, (image_height, image_width)))
        x.dtype = np.uint8
        y = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        y = cv2.resize(y, (image_height, image_width))

        x= x/255.0
        y= y/255.0
        data_obj['image'].append(x)
        data_obj['mask'].append(y)

    return data_obj

In [2]:
data = load_data(dpath, data, classes[0])
ln = len(data['image'])
print("Number of benign tumor images", ln)
data = load_data(dpath, data, classes[1])
print("Number of malignant tumor images", len(data['image'])-ln)
print("Total images for segmentation", len(data['image']))

NameError: name 'load_data' is not defined

# Visualization

In [ ]:
def visualize_example(data, index = None):
    if index is None:
        index = random.randint(0, len(data['image']))
    X = data['image']
    y = data['mask']
    has_mask = y[index].max() > 0

    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
    ax[0].imshow(X[index])
    if has_mask:
        ax[0].contour(y[index].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Image')

    ax[1].imshow(y[index].squeeze())
    ax[1].set_title('Mask')


In [ ]:
visualize_example(data)

In [ ]:
data['image'] = np.array(data['image'])
data['image'].shape


In [ ]:
data['mask'] = np.array(data['mask'])
data['mask'] =  np.expand_dims(data['mask'], -1)
data['mask'].shape


# Train-Test Split

In [ ]:


X_train, X_valid, y_train, y_valid = train_test_split(data['image'], data['mask'], test_size=0.2, random_state=42)
print(len(X_train), len(y_train), len(X_valid), len(y_valid))

# Data Augmentation

In [ ]:
data_gen_args = dict(
rotation_range=60,
zoom_range=[0.6,1.0],
horizontal_flip=True,
vertical_flip = True,
fill_mode="nearest",
)

In [ ]:
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
seed = 1

In [ ]:

image_datagen.fit(X_train, augment=True, seed=seed)

In [ ]:
mask_datagen.fit(y_train, augment=True, seed=seed)

# Fitting and Creating Train Generator

In [ ]:
image_generator = image_datagen.flow(
    X_train,
    batch_size = 16,
    seed=seed)
mask_generator = mask_datagen.flow(
    y_train,
    batch_size = 16,
    seed=seed)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))
for i in range(3):
    image = next(image_generator)[0]
    ax[i].imshow(image)
    ax[i].axis('off')

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))
for i in range(3):
    image = next(mask_generator)[0]
    ax[i].imshow(image)
    ax[i].axis('off')

In [ ]:
train_generator = zip(image_generator, mask_generator)


In [ ]:
tf.keras.backend.clear_session()